# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [6]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 7.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install langchain_community langchain_experimental langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [13]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=45806cede0e95bcec4a55ee0c0599becda41699ec8d2d3ef672460fbd9bdc2c6
  Stored in directory: /Users/rawan/Library/Caches/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")  
# Access the keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [4]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [ ]:
# from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
# from langchain.chains import LLMChain
# from langchain_community.utilities import SerpAPIWrapper
# from langchain_openai import OpenAI
# from langchain.prompts import PromptTemplate

# # Travel itinerary prompt
# travel_prompt = PromptTemplate.from_template(
#     "You are a travel expert AI. Create a short and practical itinerary for: {objective}."
# )

# # LLM chain for planning
# travel_chain = LLMChain(
#     llm=OpenAI(temperature=0.7, max_tokens=300),
#     prompt=travel_prompt
# )

# # Tools: search + trip planner
# tools = [
#     Tool(
#         name="Search",
#         func=SerpAPIWrapper().run,
#         description="For checking current travel info like weather, events, or bookings."
#     ),
#     Tool(
#         name="TripPlanner",
#         func=travel_chain.run,
#         description="Generates a personalized travel itinerary."
#     ),
# ]

# # Agent prompt setup
# prefix = "You are an AI assistant helping plan a great trip based on: {objective}. Use available tools to help."
# suffix = "Next task: {task}\n{agent_scratchpad}"

# # Final agent prompt
# prompt = ZeroShotAgent.create_prompt(
#     tools=tools,
#     prefix=prefix,
#     suffix=suffix,
#     input_variables=["objective", "context", "task", "agent_scratchpad"]
# )


/var/folders/c1/6mnj5v850svfvqx_0lfxbfzm0000gn/T/ipykernel_32372/2004336693.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  travel_chain = LLMChain(


In [5]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

# Prompt for generating fitness plans
fitness_prompt = PromptTemplate.from_template(
    "You are a certified personal trainer AI. Create a customized and achievable fitness plan based on the following goal: {objective}."
)

# LLM chain for fitness planning
fitness_chain = LLMChain(
    llm=OpenAI(temperature=0.6),
    prompt=fitness_prompt
)

# Tools: Search + Fitness Plan Generator
tools = [
    Tool(
        name="Search",
        func=SerpAPIWrapper().run,
        description="Useful for looking up workout techniques, current trends, or healthy nutrition tips."
    ),
    Tool(
        name="FitnessPlanner",
        func=fitness_chain.run,
        description="Generates personalized fitness plans or workout routines."
    ),
]

# Agent prompt setup
prefix = "You are a fitness AI assistant helping users reach their health goals. Use available tools to support their request: {objective}."
suffix = "Next task: {task}\n{agent_scratchpad}"

# Final agent prompt
prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "context", "task", "agent_scratchpad"]
)


/var/folders/c1/6mnj5v850svfvqx_0lfxbfzm0000gn/T/ipykernel_33293/2327863893.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  fitness_chain = LLMChain(


In [6]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/var/folders/c1/6mnj5v850svfvqx_0lfxbfzm0000gn/T/ipykernel_33293/3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [7]:
OBJECTIVE = "Build upper body strength with no gym access"

In [8]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [9]:
baby_agi({"objective": OBJECTIVE})

/var/folders/c1/6mnj5v850svfvqx_0lfxbfzm0000gn/T/ipykernel_33293/3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...

Thought: I should think about what tasks need to be done
Action: FitnessPlanner
Action Input: "Make a todo list"
Observation: 

Goal: To create a todo list for a personalized and achievable fitness plan.

1. Assess Your Current Fitness Level: Before creating a fitness plan, it is important to assess your current fitness level. This will help you determine your strengths, weaknesses, and any potential limitations. You can do this by taking a fitness test or consulting with a certified personal trainer.

2. Set Realistic Goals: Once you have assessed your fitness level, it is important to set realistic and achievable goals. This will help you stay motivated and on track with your fitness plan. Your goals should be specific, measurable, attainable, relevant, and time-bound (SMART).

3. Choose Your Exercise Routine: Based on your goals and fitness level, choose an ex

{'objective': 'Build upper body strength with no gym access'}